# Problem: Order Processing System

Design a small system to represent and validate customer orders.
Each order includes customer details, a list of items, and a total amount.

### Requirements

1. Each item has a name, quantity, and price, 
    * Price and quantity must be positive.
    * Price must be round to 2 decimal places.

2. A customer has a 
    * name
    * email address.

3. An order has:

    * a unique id

    * one customer

    * a list of items (must contain at least one)

    * an optional discount code

    * a declared total that must equal the sum of all item prices (minus 10% if discount applied)

4. The system should:

    * Round all prices to two decimals

    * Validate the total price automatically

    * Provide a computed property for total item count

# Solution

### 1. Defining the **Item** Model

An Item represents a product with:
* **name**: Item identifier (string)
* **quantity**: Number of units (must be a positive integer)
* **price**: Unit price (must be positive, automatically rounded to 2 decimal places)

In [1]:
from pydantic import BaseModel, Field, PositiveInt, field_validator

class Item(BaseModel):
    """
    Represents a single item in an order.
    
    Attributes:
        name: The item's name
        quantity: Number of units (must be positive integer)
        price: Unit price (must be positive, rounded to 2 decimals)
    """
    name: str 
    quantity: int = PositiveInt  # Ensures quantity is a positive integer
    price: float = Field(gt=0)   # Field with constraint: greater than 0
    
    @field_validator("price")  # Custom validator runs after basic type validation
    @classmethod
    def round_price(cls, v):
        """Round price to 2 decimal places for consistency"""
        return round(v, 2)

In [2]:
# Good Test: Valid item with price rounded to 2 decimal places
try:
    item_1 = Item(name="Apple", quantity=10, price=2.1279)
    print(item_1)
    # Notice: price 2.1279 is automatically rounded to 2.13
except Exception as e:
    print(f"Validation error: {e.json(indent=2)}")

name='Apple' quantity=10 price=2.13


In [3]:
# Bad Test: Invalid item with negative quantity and price
try:
    item_2 = Item(name="Huy", quantity=-1.5, price=-5)
    print(item_2)
except Exception as e:
    # Pydantic catches multiple validation errors:
    # 1. quantity must be integer (not float with fractional part)
    # 2. price must be greater than 0
    print(f"Validation error: {e.json(indent=2)}")

Validation error: [
  {
    "type": "int_from_float",
    "loc": [
      "quantity"
    ],
    "msg": "Input should be a valid integer, got a number with a fractional part",
    "input": -1.5,
    "url": "https://errors.pydantic.dev/2.12/v/int_from_float"
  },
  {
    "type": "greater_than",
    "loc": [
      "price"
    ],
    "msg": "Input should be greater than 0",
    "input": -5,
    "ctx": {
      "gt": 0.0
    },
    "url": "https://errors.pydantic.dev/2.12/v/greater_than"
  }
]


In [4]:
from pydantic import EmailStr

class Customer(BaseModel):
    """
    Represents a customer in the order system.
    
    Attributes:
        name: Customer's full name
        email: Email address (validated for proper format)
    """
    name: str
    email: EmailStr  # Automatically validates email format (requires @ symbol, domain, etc.)

### 2. Defining the **Customer** Model

A Customer represents a buyer with:
* **name**: Customer's full name
* **email**: Email address (automatically validated for correct format)

In [5]:
# Good Test: Valid customer with properly formatted email
try:
    customer_1 = Customer(name="Huy", email="example@mail.com")
    print(customer_1)
except Exception as e:
    print(f"Validation error: {e.json(indent=2)}")

name='Huy' email='example@mail.com'


In [6]:
# Bad Test: Invalid email without @ symbol
try:
    customer_2 = Customer(name="Huy", email="example.com")
    print(customer_2)
except Exception as e:
    # EmailStr validation catches missing @ symbol
    print(f"Validation error: {e.json(indent=2)}")

Validation error: [
  {
    "type": "value_error",
    "loc": [
      "email"
    ],
    "msg": "value is not a valid email address: An email address must have an @-sign.",
    "input": "example.com",
    "ctx": {
      "reason": "An email address must have an @-sign."
    },
    "url": "https://errors.pydantic.dev/2.12/v/value_error"
  }
]


### 3. Defining the **Order** Model

An Order represents a complete transaction with:
* **id**: Unique order identifier (positive integer)
* **customer**: Customer name
* **items**: List of items (must contain at least one item)
* **discount_code**: Optional discount code (10% off when provided)
* **total**: Declared total amount (validated against calculated subtotal)

**Computed Properties:**
* **item_count**: Automatically calculated total quantity of all items

**Validation:**
* Verifies the declared total matches the calculated total (with discount applied if applicable)

In [7]:
from pydantic import computed_field, model_validator
from typing import Optional

class Order(BaseModel):
    """
    Represents a customer order with automatic total validation.
    
    Attributes:
        id: Unique order identifier
        customer: Customer name
        items: List of items (minimum 1 required)
        discount_code: Optional discount code (applies 10% discount)
        total: Declared total (must match calculated subtotal)
    """
    id: PositiveInt
    customer: str
    items: list[Item] = Field(min_length=1)  # Require at least one item
    discount_code: str | None = None         # Optional discount code
    total: float                             # User-supplied total (will be validated)

    @computed_field
    @property
    def item_count(self) -> int:
        """
        Computed property: Total quantity of all items in the order.
        This field is automatically included when serializing the model.
        """
        return sum(i.quantity for i in self.items)

    @model_validator(mode="after")
    def check_total(self):
        """
        Validates that the declared total matches the calculated subtotal.
        Runs after all fields have been validated and the model is constructed.
        
        Calculation:
        1. Sum all (price x quantity) for each item
        2. If discount_code exists, apply 10% discount (multiply by 0.9)
        3. Compare with declared total (rounded to 2 decimals)
        """
        subtotal = sum(i.price * i.quantity for i in self.items)
        if self.discount_code:
            subtotal = round(subtotal * 0.9, 2)
        if round(subtotal, 2) != round(self.total, 2):
            raise ValueError(f"Total mismatch: expected {subtotal:.2f}, got {self.total:.2f}")
        return self

In [8]:
# Good Test: Valid order without discount
try:
    order_1 = Order(
        id=1,
        customer="John Doe",
        items=[
            Item(name="Apple", quantity=3, price=1.5),
            Item(name="Banana", quantity=6, price=0.5)
        ],
        discount_code=None,  # No discount code provided
        total=7.5  # Subtotal: (3×1.5) + (6×0.5) = 4.5 + 3.0 = 7.5
    )
    print(order_1)
    print("*" * 40)
    print(f"Item count: {order_1.item_count}")  # Total quantity: 3 + 6 = 9

except Exception as e:
    print(e.json(indent=4))

id=1 customer='John Doe' items=[Item(name='Apple', quantity=3, price=1.5), Item(name='Banana', quantity=6, price=0.5)] discount_code=None total=7.5 item_count=9
****************************************
Item count: 9


In [9]:
# Bad Test: Order with empty items list
try:
    order_2 = Order(
        id=1,
        customer="John Doe",
        items=[],  # Invalid: at least one item is required
        total=9
    )
    print(order_2)
except Exception as e:
    # Pydantic catches that items list doesn't meet min_length=1 requirement
    print(e.json(indent=4))

[
    {
        "type": "too_short",
        "loc": [
            "items"
        ],
        "msg": "List should have at least 1 item after validation, not 0",
        "input": [],
        "ctx": {
            "field_type": "List",
            "min_length": 1,
            "actual_length": 0
        },
        "url": "https://errors.pydantic.dev/2.12/v/too_short"
    }
]


In [10]:
# Bad Test: Order with incorrect total (when discount is applied)
try:
    order_3 = Order(
        id=1,
        customer="John Doe",
        items=[
            Item(name="Apple", quantity=3, price=1.5),
            Item(name="Banana", quantity=6, price=0.5)
        ],
        discount_code="SUMMER10",  # 10% discount applied
        total=9  # Incorrect! Should be 7.5 × 0.9 = 6.75
    )
    print(order_3)
    print("*" * 40)
    print(f"Item count: {order_3.item_count}")
except Exception as e:
    # model_validator catches mismatch between declared total and calculated total
    # Calculation: Subtotal = 7.5, with discount = 7.5 × 0.9 = 6.75 (not 9)
    print(e.json(indent=4))

[
    {
        "type": "value_error",
        "loc": [],
        "msg": "Value error, Total mismatch: expected 6.75, got 9.00",
        "input": {
            "id": 1,
            "customer": "John Doe",
            "items": [
                {
                    "name": "Apple",
                    "quantity": 3,
                    "price": 1.5
                },
                {
                    "name": "Banana",
                    "quantity": 6,
                    "price": 0.5
                }
            ],
            "discount_code": "SUMMER10",
            "total": 9
        },
        "ctx": {
            "error": "Total mismatch: expected 6.75, got 9.00"
        },
        "url": "https://errors.pydantic.dev/2.12/v/value_error"
    }
]


# Model Serialization

## Serializing Models (Converting to JSON)

Pydantic provides two methods for serialization:

1. **`model_dump()`**: Returns a Python dictionary
   - Use when you need to work with the data in Python
   - Easy to manipulate, pass to other functions, or convert to other formats

2. **`model_dump_json()`**: Returns a JSON string
   - Use when you need to save to disk, send over API, or store in database
   - Already formatted and ready for transmission

In [11]:
# Serialize to Python dictionary
a = order_1.model_dump()
print(type(a))  # <class 'dict'>
a  # Notice: item_count is included (computed field)

<class 'dict'>


{'id': 1,
 'customer': 'John Doe',
 'items': [{'name': 'Apple', 'quantity': 3, 'price': 1.5},
  {'name': 'Banana', 'quantity': 6, 'price': 0.5}],
 'discount_code': None,
 'total': 7.5,
 'item_count': 9}

In [12]:
# Serialize to JSON string
b = order_1.model_dump_json()
print(type(b))  # <class 'str'>
b  # Ready to save to file or send via API

<class 'str'>


'{"id":1,"customer":"John Doe","items":[{"name":"Apple","quantity":3,"price":1.5},{"name":"Banana","quantity":6,"price":0.5}],"discount_code":null,"total":7.5,"item_count":9}'

## Deserializing Models (Loading Back)

Pydantic provides corresponding methods for deserialization:

1. **`model_validate(dict)`**: Creates a model instance from a Python dictionary
2. **`model_validate_json(json_str)`**: Creates a model instance from a JSON string

Both methods perform full validation, ensuring data integrity.

In [13]:
# Deserialize from Python dictionary
# All validations run again to ensure data integrity
Order.model_validate(a)

Order(id=1, customer='John Doe', items=[Item(name='Apple', quantity=3, price=1.5), Item(name='Banana', quantity=6, price=0.5)], discount_code=None, total=7.5, item_count=9)

In [14]:
# Deserialize from JSON string
# Parses JSON and validates all fields
Order.model_validate_json(b)

Order(id=1, customer='John Doe', items=[Item(name='Apple', quantity=3, price=1.5), Item(name='Banana', quantity=6, price=0.5)], discount_code=None, total=7.5, item_count=9)

# Key Concepts Summary

## Validation Tools

1. **Built-in Validators** (Quick validation without custom code):
   - `Field(gt=0)`: Greater than constraint
   - `Field(min_length=1)`: Minimum list/string length
   - `PositiveInt`: Ensures positive integers
   - `EmailStr`: Validates email format

2. **`@field_validator(field_name)`**: Custom field validation
   - Runs **after** basic type validation
   - **MUST** be a `@classmethod`
   - Receives the field value, returns the processed value
   - Use for custom transformations (e.g., rounding, formatting)

3. **`@model_validator(mode="after")`**: Cross-field validation
   - Runs **after** all fields are validated and the model is constructed
   - **MUST NOT** be a classmethod (use `self` to access fields)
   - Receives the complete model instance
   - Use for validations that depend on multiple fields (e.g., total calculation)

4. **`@computed_field`**: Computed properties
   - Adds calculated fields to the model
   - Automatically included when serializing (dump)
   - Acts like a read-only property
   - Use for derived values (e.g., item count, subtotals)

## Important Notes on Validators

**Common Confusion:**

```python
@field_validator("price")          # MUST be @classmethod
@classmethod
def validate_price(cls, v):
    return round(v, 2)

@model_validator(mode="before")    # MUST be @classmethod
@classmethod
def validate_before(cls, data):
    return data

@model_validator(mode="after")     # MUST NOT be @classmethod (use self)
def validate_after(self):
    # Can access self.field_name
    return self
```

## Serialization/Deserialization Flow

```
Model Instance → model_dump() → Python Dict
              → model_dump_json() → JSON String

Python Dict → model_validate() → Model Instance
JSON String → model_validate_json() → Model Instance
```